# Enable:
* p100 GpU
* Add-ons -> Secrets -> Check all
* presistance -> Files only
*  internet -> on

In [1]:
print('hi')

hi


In [ ]:
# !ls /kaggle/input/gpt-v5-512/model_checkpoints/

In [2]:
# !rm -rf /kaggle/working/model_checkpoints
# !mkdir /kaggle/working/model_checkpoints
# !cp /kaggle/input/sebastian-gpt2/model_pg_330000_steps.pth /kaggle/working/model_checkpoints/




from kaggle_secrets import UserSecretsClient
import os
user_secrets = UserSecretsClient()
os.environ['KAGGLE_KEY'] = user_secrets.get_secret("KAGGLE_KEY")
os.environ['KAGGLE_USERNAME'] = user_secrets.get_secret("KAGGLE_USERNAME")



In [3]:
!ls

functions.py
hf_cache
model_checkpoints
prepare_dataset.py
pretraining_bells_n_whistles_modified_training_loop.py
pretraining_bells_n_whistles.py
pretraining_simple.py
previous_chapters.py
__pycache__


In [4]:
!rm -rf /kaggle/working/model_checkpoints/
# !kaggle kernels output aanandaint/gpt-v5-512 -p /kaggle/working/

# !rm -rf /kaggle/working/hf_cache
# !pip install datasets --quiet

# # copy previously trained model
# # !mkdir model_checkpoints
# # !cp /kaggle/input/sebastian-v5-512/model_checkpoints/model_pg_115000_steps.pth model_checkpoints/
# # !cp /kaggle/input/sebastian-gpt2/model_pg_epoch_0.pth model_checkpoints/model_pg_150000_steps.pth

# training code
!cp -r /kaggle/input/sebastian-gpt2/*.py ./
# print('done')

In [11]:
!ls /kaggle/working/
!mkdir /kaggle/working/model_checkpoints

functions.py
hf_cache
model_checkpoints
prepare_dataset.py
pretraining_bells_n_whistles_modified_training_loop.py
pretraining_bells_n_whistles.py
pretraining_simple.py
previous_chapters.py
__pycache__
mkdir: cannot create directory ‘/kaggle/working/model_checkpoints’: File exists


In [12]:
!cp /kaggle/input/sebastian-gpt2/model_pg_380000_steps.pth model_checkpoints/

In [13]:
!ls /kaggle/working/model_checkpoints

model_pg_380000_steps.pth


In [14]:
from kaggle_secrets import UserSecretsClient
import os
user_secrets = UserSecretsClient()

os.environ['HF_TOKEN'] = user_secrets.get_secret("HF_TOKEN")
print('done')

done


In [ ]:
# source: https://github.com/rasbt/LLMs-from-scratch/blob/main/appendix-D/01_main-chapter-code/appendix-D.ipynb
!python pretraining_bells_n_whistles_modified_training_loop.py \
  --n_epochs 1\
  --batch_size 10 \
  --output_dir model_checkpoints \
  --eval_freq 2000 \
  --save_ckpt_freq_steps 5000 \
  --context_length 512

Loading existing model: model_checkpoints/model_pg_380000_steps.pth
----------------------------------------------------------------------
train_losses: <class 'list'>  len: 191
val_losses: <class 'list'>  len: 191
track_tokens_seen: <class 'list'>  len: 191
track_lrs: <class 'list'>  len: 379994
previous epochs: <class 'int'> 99694
previous global step: 380001 
 previous epochs: 99694

----------------------------------------------------------------------


In [ ]:
!rm -rf /kaggle/working/hf_cache

In [6]:
!rm -rf ./*
!ls

## load epoch_3 model from previous run

In [13]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ['KAGGLE_KEY'] = user_secrets.get_secret("KAGGLE_KEY_THIS_ACCOUNT")
os.environ['KAGGLE_USERNAME'] = user_secrets.get_secret("KAGGLE_USERNAME_THIS_ACCOUNT")

!kaggle kernels output aanandaint/gpt-v5-512 -p ./

Output file downloaded to /path/to/dest/__pycache__/functions.cpython-310.pyc
Output file downloaded to /path/to/dest/__pycache__/previous_chapters.cpython-310.pyc
Output file downloaded to /path/to/dest/functions.py
Output file downloaded to /path/to/dest/model_checkpoints/losses.pdf
Output file downloaded to /path/to/dest/model_checkpoints/model_pg_420000_steps.pth
Output file downloaded to /path/to/dest/model_checkpoints/model_pg_epoch_140152.pth
Output file downloaded to /path/to/dest/prepare_dataset.py
Output file downloaded to /path/to/dest/pretraining_bells_n_whistles.py
Output file downloaded to /path/to/dest/pretraining_bells_n_whistles_modified_training_loop.py
Output file downloaded to /path/to/dest/pretraining_simple.py
Output file downloaded to /path/to/dest/previous_chapters.py
Kernel log downloaded to /path/to/dest/gpt-v5-512.log 


In [22]:
!mv /kaggle/working/model_checkpoints/model_pg_epoch_140152.pth /kaggle/working/model_checkpoints/model_pg_epoch_3.pth

## push model checkpoint itself to huggingface
* we might need it later

In [105]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import HfApi

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")


api = HfApi()

files_to_upload = ["model_checkpoints/model_pg_epoch_3.pth","model_checkpoints/losses.pdf"]

for file in files_to_upload:
    api.upload_file(
        path_or_fileobj=file,    # e.g. 'model_checkpoints/model_pg_20_steps.pth'
        path_in_repo=file,       # e.g. 'model_checkpoints/model_pg_20_steps.pth'
        repo_id="Aananda-giri/gpt2-nepali",
        repo_type="model",
        token=hf_token
    )

model_pg_epoch_3.pth:   0%|          | 0.00/1.96G [00:00<?, ?B/s]

## load model from model_checkpoints/model_pg_epoch_3.pth
* get file `model_code.py` file from github/aananda-giri/3_gpt2-nepali/2_inference

In [23]:
from transformers import PreTrainedTokenizerFast
import torch
import torch.nn as nn
from model_code import GPTModel, GPT_CONFIG_124M, generate

# load the model
# ----------------------------

model = GPTModel(GPT_CONFIG_124M)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

checkpoint = torch.load('/kaggle/working/model_checkpoints/model_pg_epoch_3.pth', weights_only=False)

# modified (added model loading code)
model.load_state_dict(checkpoint["model_state_dict"])


# load the tokenizer
# ----------------------------
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("Aananda-giri/NepaliBPE")


# generate a sample
# ----------------------------
prompt = "रामले भात"

text = generate(  # function uses `with torch.no_grad()` internally already
        model=model,
        prompt=prompt,
        tokenizer=tokenizer,
        max_new_tokens=50,
        top_k=3,
        temperature=3.0
    )
text

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.53M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

रामले भात पकाउँदै थिए । तर उनी खाना खान तयार भएनन् । खाना खान तयार भएनन् । खाना खान तयार भएनन् । खाना खान तयार भएनन् । खाना खान तयार भएनन् । खाना खान तयार भएनन् । खाना खान तयार भएनन् । खाना खान तयार भएनन् । खाना खान तयार भएनन्


## push model and tokenizers to hub

In [24]:
# push the model to hub
# ----------------------
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HF_TOKEN")

model.push_to_hub("Aananda-giri/GPT2-Nepali", token=hf_token)

model.safetensors:   0%|          | 0.00/661M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Aananda-giri/GPT2-Nepali/commit/ff1816046bd6f32075d09c21bfb9df756fa6646c', commit_message='Push model using huggingface_hub.', commit_description='', oid='ff1816046bd6f32075d09c21bfb9df756fa6646c', pr_url=None, pr_revision=None, pr_num=None)

In [25]:
# Push tokenizer to hub
# ----------------------
tokenizer.push_to_hub("Aananda-giri/GPT2-Nepali", token=hf_token)

README.md:   0%|          | 0.00/373 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Aananda-giri/GPT2-Nepali/commit/566657d07bd87573e676795b310c6be654bd31fa', commit_message='Upload tokenizer', commit_description='', oid='566657d07bd87573e676795b310c6be654bd31fa', pr_url=None, pr_revision=None, pr_num=None)

## load model from hub

In [13]:
%%writefile previous_chapte.py
# Copyright (c) Sebastian Raschka under Apache License 2.0 (see LICENSE.txt).
# Source for "Build a Large Language Model From Scratch"
#   - https://www.manning.com/books/build-a-large-language-model-from-scratch
# Code: https://github.com/rasbt/LLMs-from-scratch
#
# This file collects all the relevant code that we covered thus far
# throughout Chapters 2-5.

import json
import os
import urllib

import numpy as np
import tensorflow as tf
import torch
import torch.nn as nn
from tqdm import tqdm

from huggingface_hub import PyTorchModelHubMixin
from transformers import PreTrainedTokenizerFast

GPT_CONFIG_124M = {
    "vocab_size": 50000,     # Vocabulary size
    "context_length": 1024,  # Context length
    "emb_dim": 768,          # Embedding dimension
    "n_heads": 12,           # Number of attention heads
    "n_layers": 12,          # Number of layers
    "drop_rate": 0.1,        # Dropout rate
    "qkv_bias": False        # Query-key-value bias
}

#####################################
# Chapter 3
#####################################
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by n_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)  # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.reshape(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec


#####################################
# Chapter 4
#####################################
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift


class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))


class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)


class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)   # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed-forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x


class GPTModel(nn.Module,
        PyTorchModelHubMixin, # modified to push the model to the hub (https://huggingface.co/docs/hub/en/models-uploading#upload-a-pytorch-model-using-huggingfacehub)
        repo_url="https://huggingface.co/Aananda-giri/GPT2-Nepali/",
        pipeline_tag="text-generation",
        ):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits


#####################################
# Chapter 5
#####################################
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text)
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)  # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0)  # remove batch dimension
    return tokenizer.decode(flat.tolist())

def load_model_n_tokenizer():
    model = GPTModel.from_pretrained("Aananda-giri/GPT2-Nepali")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    tokenizer = PreTrainedTokenizerFast.from_pretrained("Aananda-giri/NepaliBPE")
    return model, tokenizer

def generate(
    model,
    prompt,
    tokenizer,
    max_new_tokens,
    temperature=0.7,
    top_k=50,
    top_p=None,  # New parameter for nucleus sampling
    eos_id=None,
    repetition_penalty=1.2,
    penalize_len_below=50
):
    context_size = GPT_CONFIG_124M['context_length']
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    idx = text_to_token_ids(prompt, tokenizer).to(device)
    
    if not eos_id:
        encoded_endoftext = tokenizer.encode("<|endoftext|>")
        eos_id = encoded_endoftext[0] if encoded_endoftext else None

    token_freq = {}

    for step in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]

        # Apply repetition penalty
        for token_id in idx[0].tolist():
            if token_id in token_freq:
                logits[0, token_id] /= repetition_penalty
            else:
                token_freq[token_id] = 1
        
        # Penalize EOT token for shorter sequences
        if eos_id is not None and step < penalize_len_below:
            logits[0, eos_id] /= (penalize_len_below - step) / penalize_len_below

        # Apply temperature scaling
        if temperature > 0.0:
            logits = logits / temperature

        # Convert logits to probabilities
        probs = torch.softmax(logits, dim=-1)

        # Apply top-p (nucleus) sampling if specified
        if top_p is not None:
            sorted_probs, sorted_indices = torch.sort(probs, descending=True)
            cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
            
            # Remove tokens with cumulative probability above the threshold
            sorted_indices_to_remove = cumulative_probs > top_p
            # Shift the indices to the right to keep also the first token above the threshold
            sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
            sorted_indices_to_remove[..., 0] = 0

            # Create a mask for indices to remove
            indices_to_remove = sorted_indices_to_remove.scatter(dim=-1, index=sorted_indices, src=sorted_indices_to_remove)
            probs = probs.masked_fill(indices_to_remove, 0.0)
            
            # Renormalize probabilities
            probs = probs / probs.sum(dim=-1, keepdim=True)

        # If top_p is None, apply top-k sampling
        elif top_k is not None:
            top_probs, top_indices = torch.topk(probs, top_k)
            probs = torch.zeros_like(probs).scatter_(-1, top_indices, top_probs)
            # Renormalize probabilities
            probs = probs / probs.sum(dim=-1, keepdim=True)

        # Sample from the filtered distribution
        if temperature > 0.0:
            idx_next = torch.multinomial(probs, num_samples=1)
        else:
            idx_next = torch.argmax(probs, dim=-1, keepdim=True)

        if idx_next == eos_id:
            break

        idx = torch.cat((idx, idx_next), dim=1)
        text = token_ids_to_text(idx, tokenizer)

    # print('done generating!')
    return text


Writing previous_chapte.py


In [18]:
# %%timeit
from previous_chapter import load_model_n_tokenizer, generate

# Download weights and load model weights
model, tokenizer = load_model_n_tokenizer()
model.eval()

prompt = "रामले भात"

text = generate(  # function uses `with torch.no_grad()` internally already
        model=model,
        prompt=prompt,
        tokenizer=tokenizer,
        max_new_tokens=50,
        top_p=0.9,# top p sampling is prefered over top k if top_p != None
        top_k=3,
        temperature=0.7,
        repetition_penalty=1.2,  # New parameter: Repetition penalty factor
        penalize_len_below=50  # New parameter: Minimum content length for penalizing EOT token.
    )
print(text)

रामले भात पकाएर खान थाले । तर उनी भोकै थिएनन् । भोकले मरिन्छ कि भन्ने डर थियो उनले भने खानै नपाइ किन मरिन्छ र
